In [5]:
from pathlib import Path
import sys, importlib

sys.path.insert(0, str(Path("..").resolve())) 

import src.data.itch as itch                     
importlib.reload(itch)         
                    
from src.data.itch import load_lobster_top1_day

root = Path("../data/raw/itch/AAPL")
df_itch = load_lobster_top1_day(root, "AAPL")

print(df_itch.shape)
display(df_itch.head())
display(df_itch[["spread"]].describe())


(400391, 7)


,time,bid,bid_size,ask,ask_size,mid,spread
0,34200.004241,5853300.0,18.0,5859400.0,200.0,5856350.0,6100.0
1,34200.004261,5853300.0,18.0,5859400.0,200.0,5856350.0,6100.0
2,34200.004447,5853300.0,18.0,5859400.0,200.0,5856350.0,6100.0
3,34200.025552,5853300.0,18.0,5859100.0,18.0,5856200.0,5800.0
4,34200.025580,5853300.0,18.0,5859100.0,18.0,5856200.0,5800.0


,spread
count,400391.000000
mean,1534.889396
std,741.575858
min,100.000000
25%,1000.000000
50%,1500.000000
75%,2000.000000
max,9200.000000


In [6]:
scale = 10_000  # LOBSTER scale
q = df_itch.copy()
q["bid$"] = q["bid"] / scale
q["ask$"] = q["ask"] / scale
q["mid$"] = q["mid"] / scale
q["spread$"] = q["spread"] / scale
q.head()

,time,bid,bid_size,ask,ask_size,mid,spread,bid$,ask$,mid$,spread$
0,34200.004241,5853300.0,18.0,5859400.0,200.0,5856350.0,6100.0,585.33,585.94,585.635,0.61
1,34200.004261,5853300.0,18.0,5859400.0,200.0,5856350.0,6100.0,585.33,585.94,585.635,0.61
2,34200.004447,5853300.0,18.0,5859400.0,200.0,5856350.0,6100.0,585.33,585.94,585.635,0.61
3,34200.025552,5853300.0,18.0,5859100.0,18.0,5856200.0,5800.0,585.33,585.91,585.620,0.58
4,34200.025580,5853300.0,18.0,5859100.0,18.0,5856200.0,5800.0,585.33,585.91,585.620,0.58


In [7]:
import pandas as pd

q1s = (q.set_index(pd.to_timedelta(q["time"], unit="s"))
         [["bid$","ask$","mid$","bid_size","ask_size","spread$"]]
         .resample("1S")
         .last()
         .ffill())

q1s["imbalance"] = (q1s["bid_size"] - q1s["ask_size"]) / (q1s["bid_size"] + q1s["ask_size"])
q1s["ret_1s"] = q1s["mid$"].pct_change(1)
q1s["ret_5s"] = q1s["mid$"].pct_change(5)
q1s["rv_30s"] = (q1s["ret_1s"].rolling(30).std() * (30 ** 0.5)).fillna(0.0)
q1s = q1s.dropna()
q1s.head()


/var/folders/xx/w12875ds6blcc9p0ljq82tdw0000gn/T/ipykernel_17866/1605847366.py:5: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  .resample("1S")


,bid$,ask$,mid$,bid_size,ask_size,spread$,imbalance,ret_1s,ret_5s,rv_30s
time,,,,,,,,,,
0 days 09:30:05.004241176,585.48,585.67,585.575,14.0,18.0,0.19,-0.125000,-0.000026,-0.000393,0.0
0 days 09:30:06.004241176,585.48,585.68,585.580,229.0,396.0,0.20,-0.267200,0.000009,-0.000068,0.0
0 days 09:30:07.004241176,585.48,585.68,585.580,229.0,396.0,0.20,-0.267200,0.000000,0.000009,0.0
0 days 09:30:08.004241176,585.49,585.68,585.585,100.0,398.0,0.19,-0.598394,0.000009,0.000009,0.0
0 days 09:30:09.004241176,585.44,585.66,585.550,66.0,2.0,0.22,0.941176,-0.000060,-0.000068,0.0


In [8]:
H = 5 
q1s["mid_fwd"] = q1s["mid$"].shift(-H)
q1s["y"] = (q1s["mid_fwd"] - q1s["mid$"]).apply(lambda x: 1 if x > 0 else 0)
features = ["imbalance","spread$","rv_30s","ret_1s"]
data = q1s.dropna(subset=features + ["y"]).copy()
X = data[features].values
y = data["y"].values
X.shape, y.shape


((23395, 4), (23395,))

In [9]:
%pip install -U scikit-learn

import sklearn
import numpy as np
from sklearn.linear_model import LogisticRegression
print("sklearn:", sklearn.__version__)


Note: you may need to restart the kernel to use updated packages.
sklearn: 1.7.2


In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=0.25, shuffle=False)  
clf = LogisticRegression(max_iter=1000)
clf.fit(Xtr, ytr)
proba = clf.predict_proba(Xte)[:,1]
print("AUC:", roc_auc_score(yte, proba))


AUC: 0.5061823377841365


In [11]:
H = 5
q = q.sort_values("time").reset_index(drop=True)
q["mid_fwd"] = q["mid"].shift(-H)
q["ret"] = (q["mid_fwd"] - q["mid"]) / q["mid"]
q["y"] = (q["ret"] > 0).astype(int)
q = q.dropna(subset=["y"])

q["spreads"] = (q["ask"] - q["bid"])             # in ticks or dollars
q["imb"] = (q["bid_size"] - q["ask_size"]) / (q["bid_size"] + q["ask_size"])
q["dmid1"] = q["mid"].diff()
q["dmid2"] = q["mid"].diff(2)
q["roll_vol_1s"] = q["dmid1"].rolling(10, min_periods=3).std()
q["roll_vol_5s"] = q["dmid1"].rolling(50, min_periods=10).std()
X = q[["imb", "spreads", "dmid1", "dmid2", "roll_vol_1s", "roll_vol_5s"]].fillna(0.0)
y = q["y"].astype(int)

In [ ]:
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

tscv = TimeSeriesSplit(n_splits=5)
aucs = []

clf = RandomForestClassifier(n_estimators=300, max_depth=6, random_state=0, n_jobs=-1) 

for tr_idx, te_idx in tscv.split(X):
    Xtr, Xte = X.iloc[tr_idx], X.iloc[te_idx]
    ytr, yte = y.iloc[tr_idx], y.iloc[te_idx]

    clf.fit(Xtr, ytr)
    proba = clf.predict_proba(Xte)[:, 1]
    aucs.append(roc_auc_score(yte, proba))

print("AUC mean:", np.mean(aucs))
print("Per-fold AUCs:", aucs)

clf.fit(X, y)
X_current = X.iloc[[-1]]         

p_up = float(clf.predict_proba(X_current)[:, 1][0])  
qshare = float(np.clip((p_up - 0.5) * 4, 0.0, 1.0))   

print(f"p_up={p_up:.3f}, qshare={qshare:.3f}")


AUC mean: 0.5987813828546367
Per-fold AUCs: [0.5718931915694615, 0.5881732493606334, 0.6179914708819588, 0.6016484114590723, 0.6142005910020576]
p_up=0.247, qshare=0.000


In [14]:
q.head()


,time,bid,bid_size,ask,ask_size,mid,spread,bid$,ask$,mid$,spread$,mid_fwd,ret,y,spreads,imb,dmid1,dmid2,roll_vol_1s,roll_vol_5s
0,34200.004241,5853300.0,18.0,5859400.0,200.0,5856350.0,6100.0,585.33,585.94,585.635,0.61,5856200.0,-0.000026,0,6100.0,-0.834862,NaN,NaN,NaN,NaN
1,34200.004261,5853300.0,18.0,5859400.0,200.0,5856350.0,6100.0,585.33,585.94,585.635,0.61,5856200.0,-0.000026,0,6100.0,-0.834862,0.0,NaN,NaN,NaN
2,34200.004447,5853300.0,18.0,5859400.0,200.0,5856350.0,6100.0,585.33,585.94,585.635,0.61,5856200.0,-0.000026,0,6100.0,-0.834862,0.0,0.0,NaN,NaN
3,34200.025552,5853300.0,18.0,5859100.0,18.0,5856200.0,5800.0,585.33,585.91,585.620,0.58,5856200.0,0.000000,0,5800.0,0.000000,-150.0,-150.0,86.60254,NaN
4,34200.025580,5853300.0,18.0,5859100.0,18.0,5856200.0,5800.0,585.33,585.91,585.620,0.58,5856200.0,0.000000,0,5800.0,0.000000,0.0,-150.0,75.00000,NaN


In [15]:
# replace this with your actual DataFrame variable name
quotes_enriched = q  # e.g., the df that has time, bid, ask, mid$, etc.

from pathlib import Path
DATA_DIR = (Path.cwd() / "data")  # if you're inside notebooks/, this is notebooks/data
DATA_DIR.mkdir(parents=True, exist_ok=True)

# Save both CSV (universal) and Pickle (preserves dtypes precisely)
quotes_enriched.to_csv(DATA_DIR / "quotes_bucketed.csv", index=False)
quotes_enriched.to_pickle(DATA_DIR / "quotes_bucketed.pkl")
print("Wrote:", DATA_DIR / "quotes_bucketed.csv")


Wrote: /Users/krithiktoure/Desktop/DS Project 2/exec-engine/notebooks/data/quotes_bucketed.csv
